# Modellering av smittespredning

Smittespredning kan simuleres ved hjelp av matematiske modeller. Denne artikkelen vil ta for seg et par ulike modeller, og til slutt sammenligne dem med ekte data.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider

plt.rcParams["figure.figsize"] = (15, 5)

weeks = 48
p = 157759

influensa_data = np.loadtxt("influensa.txt", delimiter=",", dtype=int)

## Modell 1

En enkel måte å simulere smittespredning på, er å si at antallet syke ved et tidspunkt $t$, er gitt ved antallet syke ved tidspunktet $t - 1$, pluss hvor mange som blir smittet i intervallet mellom de to tidspunktene. Matematisk kan dette representeres som den numeriske modellen under.

### Numerisk modell $I_{t+1} = I_n + a I_n$

$I$ er antall smittede, og variabelen $a$ er produktet av kontaktraten og den prosentvise smittsomheten til sykdommen.

Det er mulig å lage en kontinuerlig modell for smitten.

### Kontinuerlig modell $I'(t) = a I(t)$  

Man kan bruke den kontinuerlige modellen for å finne en funksjon for $I$ gitt ved $t$.  
Dersom man setter antall syke i uke 0 til 1, kan man finne en funksjon for $I(t)$ ved å løse startverdiproblemet $I'(t) = a I(t), \hspace{0.3cm} I(0) = 1$  
Man ender da opp med $I(t) = e^{at}$

Programmene som følger bruker en kontaktrate $a$ som er gitt i per uke, og følger en populasjon på 157758 individer over en periode på 48 uker. Siden man må løse et startverdiproblet for å finne en kontinuerlig funksjon for $I(t)$ vil programmene for kontinuerlige modeller her alltid starte med 1 smittet person i uke 0.

#### Program for numerisk modell:

In [2]:
@interact(I_0 = FloatSlider(value=1, min=1, max=1000, step=1), a = FloatSlider(value=0.3, min=0, max=1, step=0.01))
def simulate(I_0, a):
    
    I = np.array([I_0] * weeks, dtype=float)
    t = np.arange(0, weeks, 1)

    for n in t[:-1]:
        I[n+1] = I[n] + a*I[n]

    plt.gca().set_ylim([0, p*1.1])

    plt.plot(t, I)
    plt.legend(["Antall syke"], loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xticks(np.arange(0, weeks + 1, 1))
    plt.title("Smittespredning")
    plt.xlabel("Uke")
    plt.ylabel("Befolkning")
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='I_0', max=1000.0, min=1.0, step=1.0), FloatSlider(va…

### Program for kontinuerlig modell:

In [3]:
@interact(a = FloatSlider(value=0.3, min=0, max=1, step=0.01))
def simulate(a):

    I = np.array([1] * weeks, dtype=float)
    t = np.arange(0, weeks, 1)

    for n in t[1:]:
        I[n] = np.e**(a*n)

    plt.gca().set_ylim([0, p*1.1])
    
    plt.plot(t, I)
    plt.legend(["Antall syke"], loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xticks(np.arange(0, weeks + 1, 1))
    plt.title("Smittespredning")
    plt.xlabel("Uke")
    plt.ylabel("Befolkning")
    plt.show()


interactive(children=(FloatSlider(value=0.3, description='a', max=1.0, step=0.01), Output()), _dom_classes=('w…

### Analyse av Modell 1

Man kan observere at Modell 1 har begrensinger. Folk blir ikke friske, og det blir ikke tatt hensyn til at antallet personer det er mulig å smitte synker når noen blir syke. Dette fører til en eksponentiell vekst, og at det er mulig å ende opp med flere syke enn det er mennesker i befolkningen. Dette gjør at modellen er dårlig ved høye smittetall.

## Modell 2: Variasjon av antall mottakelige

For å unngå problemet med det blir flere syke mennesker enn det er mennesker i befolkningen, må man sørge for at syke mennesker ikke smitter andre som allerede er syke. Dette kan oppnås ved å multiplisere kontakraten med den prosentvise andellen av befolkningen som er frisk. Man ender da opp med modellen under.

### Numerisk modell $I_{t+1} = I_n + a I_n S_n$

Man kan utrykke $S(t)$ ved $1 - \frac{I(t)}{p}$, der $p$ er total befolkning.

Også her er det mulig å lage en kontinuerlig modell.

### Kontinuerlig modell $I(t) =  a I(t) S(t)$

Funksjonen $I(t)$ kan bli utregnet ved å løse startverdiproblemet $I'(t) = a I(t) S(t), \hspace{0.3cm} I(0) = 1$  
Siden $S(t) = 1 - \frac{I(t)}{p}$, kan man skrive om startverdiproblemt til $I'(t) = a I(t)\left(1 - \frac{I(t)}{p}\right), \hspace{0.3cm} I(0) = 1$  
Løsningen på dette blir $I(t) = p\left(\frac{e^{ax}}{p+e^{ax}-1}\right)$

#### Program for numerisk modell:

In [4]:
@interact(I_0 = FloatSlider(value=1, min=1, max=1000, step=1), a = FloatSlider(value=0.4, min=0, max=1, step=0.01))
def simulate(I_0, a):
    
    I = np.array([I_0] * weeks, dtype=float)
    t = np.arange(0, weeks, 1)

    for n in t[:-1]:
        S = 1 - I[n]/p
        I[n + 1] = I[n] + a*I[n]*S


    plt.plot(t, I)
    plt.plot(t, p - I)
    plt.legend(["Antall syke", "Antall mottakelige"], loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xticks(np.arange(0, weeks + 1, 1))
    plt.title("Smittespredning")
    plt.xlabel("Uke")
    plt.ylabel("Befolkning")
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='I_0', max=1000.0, min=1.0, step=1.0), FloatSlider(va…

### Program for kontinuerlig modell:

In [5]:
@interact(a = FloatSlider(value=0.4, min=0, max=1, step=0.01))
def simulate(a):

    I = np.array([1] * weeks, dtype=float)
    t = np.arange(0, weeks, 1)

    for n in t[1:]:
        I[n] = p*np.e**(a*n)/(p+np.e**(a*n)-1)
    
    plt.plot(t, I)
    plt.plot(t, p - I)
    plt.legend(["Antall syke", "Antall mottakelige"], loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xticks(np.arange(0, weeks + 1, 1))
    plt.title("Smittespredning")
    plt.xlabel("Uke")
    plt.ylabel("Befolkning")
    plt.show()

interactive(children=(FloatSlider(value=0.4, description='a', max=1.0, step=0.01), Output()), _dom_classes=('w…

### Analyse av Modell 2

Modell 2 unngår problemet med at det blir flere syke mennesker enn det er mennesker i befolkningen. Man kan se at vekstfarten til $I$ øker i starten, men etterhvert som det blir færre mulige folk å smitte, synker vekstfarten. Folk har fremdeles ikke mulighet til å bli frisk, noe som betyr at modellen kun vil kunne brukes på sykdommer som ikke er kurerbare.

## Modell 3: Folk kan bli friske

For å legge til at folk blir friske trenger man en ny variabel, kalt bedringskoeffisient. Med denne variabelen får man den numeriske modellen under.

### Numerisk modell $I_{t+1} = I_n + a I_n S_n - b I_n$

Variabelen $b$ er bedringsraten, som er hvor stor andell av de syke som blir frisk per uke. Folk som allerede har vært syk kan ikke bli syk på nytt, noe som betyr at $S$ må bli redefinert. $S$ er nå 1 minus summen av mennesker som har vært syk og som er syk, delt på total befolkning.

Det er mulig å lage en kontinuerlig modell, men det er vanskeligere å løse startverdiproblemet siden $S$ er definert ut fra både hvor mange som er syke og hvor mange som har vært syke. Det er derfor ikke laget noe program for den kontinuerlige modellen her.

### Kontinuerlig modell $I'(t) = a I(t) S(t) - b(t)$

#### Program for numerisk modell:

In [8]:
@interact(I_0 = FloatSlider(value=1, min=1, max=1000, step=1), a = FloatSlider(value=4.02, min=3, max=5, step=0.01), b = FloatSlider(value=3.47, min=1, max=4, step=0.01))
def simulate(I_0, a, b):

    I = np.array([I_0] * weeks, dtype=float)
    t = np.arange(0, weeks, 1)

    immune = 0

    for n in t[:-1]:
        immune += b*I[n]
        S = 1 - (I[n] + immune)/p
        I[n + 1] = I[n] + a*I[n]*S - b*I[n]


    plt.plot(t, I)
    plt.plot(np.split(influensa_data, 2, 1)[0], np.split(influensa_data, 2, 1)[1])
    plt.legend(["Antallet syke basert på modellen", "Antallet faktisk syke"], loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xticks(np.arange(0, weeks + 1, 1), [i%52 for i in range(40, 40 + weeks + 1)])
    plt.title("Smittespredning")
    plt.xlabel("Uke")
    plt.ylabel("Befolkning")
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='I_0', max=1000.0, min=1.0, step=1.0), FloatSlider(va…

### Analyse av Modell 3

Ved å justere verdiene til $a$ og $b$ i modellen er det mulig å få den til å simulere influensasmitte ganske bra. $b$ med verdien 3.47 bør gi mening, da det vil tilsvare at folk gjennomsnittlig blir frisk på ca. en tredjedel av en uke. Modellen er likevel begrenset i forhold til at kontaktraten til mennesker varierer i løpet av et år. Dette kan man se tydelig på antallet faktisk syke fra uke 1 og 2.

## Vaksinering

Det er mulig å simulere vaksinering med Modell 3, men da må man redefinere $S$ enda en gang. $S$ er nå 1 - summen av de som er syke, har vært syk, og de som er vaksinerte, delt på total befolkning. Programmet under simulerer vaksinering ved at $i$ er hvor stor andel av befolkningen som er vaksinert.

#### Numerisk program med vaksinering

In [9]:
@interact(I_0 = FloatSlider(value=1, min=1, max=1000, step=1), a = FloatSlider(value=4.02, min=3, max=5, step=0.01), b = FloatSlider(value=3.47, min=1, max=4, step=0.01), i = FloatSlider(value=0.08, min=0, max=0.15, step=0.01))
def simulate(I_0, a, b, i):
    
    I = np.array([I_0] * weeks, dtype=float)
    t = np.arange(0, weeks, 1)

    immune = 0
    vaccinated = p*i

    for n in t[:-1]:
        immune += b*I[n]
        S = 1 - (I[n] + immune + vaccinated)/p
        I[n + 1] = I[n] + a*I[n]*S - b*I[n]


    plt.plot(t, I)
    plt.plot(np.split(influensa_data, 2, 1)[0], np.split(influensa_data, 2, 1)[1])
    plt.legend(["Antallet syke basert på modellen", "Antallet faktisk syke"], loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xticks(np.arange(0, weeks + 1, 1), [i%52 for i in range(40, 40 + weeks + 1)])
    plt.title("Smittespredning")
    plt.xlabel("Uke")
    plt.ylabel("Befolkning")
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='I_0', max=1000.0, min=1.0, step=1.0), FloatSlider(va…

### Analyse av vaksinering

Som man kan se på figuren ovenfor kan det å vaksinere en liten andel av befolkningen ha en veldig stor påvirkning på maksimal smitte. Modellen her sier at ved å vaksinere bare 8% av befolkningen, er det mulig å redusere den maksimale smittetoppen til nesten en syvendedel av det den originalt var.